In [2]:
import mne
import numpy as np
import pandas as pd
from scipy.signal import find_peaks

# CSVファイルからデータを読み込む
data = pd.read_csv('DP_droptime1.csv')

# EPOC Xの14ch EEGデータを取得する
channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
eeg_data = data[channels].T

# データからMNEのRawオブジェクトを作成する
info = mne.create_info(ch_names=channels, sfreq=256, ch_types='eeg')
raw = mne.io.RawArray(eeg_data, info)

# 刺激が与えられたタイミングを探す（Markerが初めて1になったインデックス）
stim_indices = data.index[data['Marker'] == 1].tolist()

# 刺激が与えられたタイミングを設定する（256Hzでのサンプル数に変換する）
events = np.array([[idx, 0, 1] for idx in stim_indices])

# エポックを作成する（刺激前1秒、刺激後2秒のエポックを作成）
epochs = mne.Epochs(raw, events, tmin=-1, tmax=2, baseline=(None, 0))

# ERPを計算する
evoked = epochs.average()

# ERPのピークを見つける（最大値のピークを探す）
peak_time = evoked.times[np.argmax(evoked.data)]

print(f'The peak is at {peak_time*1000} ms.')


Creating RawArray with float64 data, n_channels=14, n_times=256
    Range : 0 ... 255 =      0.000 ...     0.996 secs
Ready.
Not setting metadata
128 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
The peak is at -1000.0 ms.


/tmp/ipykernel_179400/1279477510.py:27: RuntimeWarning: evoked object is empty (based on less than 1 epoch)
  evoked = epochs.average()
